# Single Node Single GPU Training

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import dask

In [2]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

wandb: Currently logged in as: skirmer (use `wandb login --relogin` to force relogin)


True

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
def train_model_fit(n_epochs, base_lr, batchsize, classes):
    
    model = tf.keras.applications.ResNet50(
        include_top=True,
        weights=None,
        classes=classes)
        
    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_mg_birds', config=wbargs)
    
    # Data
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/datasets/birds/285 birds/train',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2).cache().shuffle(1000)
        
    valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '/home/jovyan/git-repos/tensorflow_ds/datasets/birds/285 birds/valid',
        image_size=(224,224),
        batch_size=batchsize
    ).prefetch(2)
   
    optimizer = keras.optimizers.Adam(lr=base_lr)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #integer labels
    start = time.time()

    model.fit(
        train_ds, 
        epochs=n_epochs, 
        validation_data=valid_ds,
        callbacks=[WandbCallback()]
    )
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    # Close your wandb run 
    wandb.run.finish()
    
    tf.keras.models.save_model(model, 'model/keras_single/')

In [5]:
model_params = {'n_epochs': 50, 
                'base_lr': .02,
                'classes': 285,
               'batchsize': 64}

wbargs = {**model_params,
    'Notes':"tf_v100_2x",
    'Tags': ['single', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet50"}

In [6]:
tester = train_model_fit(**model_params)

Found 40930 files belonging to 285 classes.
Found 1425 files belonging to 285 classes.
Epoch 1/50
640/640 [==============================] - 196s 196ms/step - loss: 6.6987 - accuracy: 0.0092 - val_loss: 5.6996 - val_accuracy: 0.0063
Epoch 2/50
640/640 [==============================] - 124s 193ms/step - loss: 5.0280 - accuracy: 0.0265 - val_loss: 4.7170 - val_accuracy: 0.0519
Epoch 3/50
640/640 [==============================] - 124s 193ms/step - loss: 4.2811 - accuracy: 0.1046 - val_loss: 4.1652 - val_accuracy: 0.1530
Epoch 4/50
640/640 [==============================] - 124s 194ms/step - loss: 3.4087 - accuracy: 0.2422 - val_loss: 3.4624 - val_accuracy: 0.2407
Epoch 5/50
640/640 [==============================] - 124s 193ms/step - loss: 2.7947 - accuracy: 0.3503 - val_loss: 4.5548 - val_accuracy: 0.2204
Epoch 6/50
640/640 [==============================] - 124s 194ms/step - loss: 2.2478 - accuracy: 0.4636 - val_loss: 2.1121 - val_accuracy: 0.4695
Epoch 7/50
640/640 [=================

epoch,49
loss,0.06213
accuracy,0.98148
val_loss,1.50092
val_accuracy,0.8014
_runtime,6279
_timestamp,1629479525
_step,49
best_val_loss,1.12216
best_epoch,13


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▆▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▁▂▃▄▅▅▆▆▇▇▇▇▇██████████████████████████
val_loss,█▆▆▅▃▂▂▂▁▁▁▁▁▂▁▂▂▂▂▂▂▂▁▃▁▂▁▂▃▂▂▁▂▂▂▂▄▁▂▂
val_accuracy,▁▁▂▃▅▆▆▆▇▇▇█▇▇█▇▇▇▇▇▇▇█▇█▇██▇███████▇█▇█
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


INFO:tensorflow:Assets written to: model/keras_single/assets
